In [1]:
# Running on Python 2.7.16 64-bit
import pandas as pd
import numpy as np

In [2]:
flow_df = pd.read_csv(r"P_VaaT_Supporting_Document_QuestionDataset_Flow.csv")
flow_df["RPT"] = flow_df["RPT"].astype(float)
flow_df["LoadSize"] = flow_df["LoadSize"].astype(int)
flow_df["RPTBasis"] = flow_df["RPTBasis"].astype(int)
flow_df["CapacityPerWeek"] = flow_df["CapacityPerWeek"].astype(int)
flow_df["WaferPerBatch"] = flow_df["WaferPerBatch"].astype(int)
flow_df.head()

,StepName,WSName,RPT,LoadSize,RPTBasis,CapacityPerWeek,WaferPerBatch
0,Step_178,Apes,0.4,1,1,22176,1
1,Step_198,Apes,0.4,1,1,22176,1
2,Step_012,Baboons,441.4,300,1,5400,300
3,Step_073,Baboons,316.3,300,1,7500,300
4,Step_187,Baboons,339.9,300,1,6900,300


In [3]:
ws_data_df = pd.read_csv(r"P_VaaT_Supporting_Document_QuestionDataset_WSData.csv")
ws_data_df["Availability%"] = ws_data_df["Availability%"].astype(float)
ws_data_df["Utilization%"] = ws_data_df["Utilization%"].astype(float)
ws_data_df["ChmCount"] = ws_data_df["ChmCount"].astype(int)
ws_data_df["ToolPrice"] = ws_data_df["ToolPrice"].astype(int)
ws_data_df["FootPrint"] = ws_data_df["FootPrint"].astype(int)
ws_data_df["LoadPort"] = ws_data_df["LoadPort"].astype(int)
ws_data_df.head()

,WSName,Availability%,Utilization%,ChmCount,ToolPrice,FootPrint,LoadPort
0,Apes,0.95,0.88,1,32000000,80,5
1,Baboons,0.91,0.81,1,1200000,90,2
2,Barracudas,0.94,0.84,1,4700000,190,5
3,Bats,0.92,0.82,3,4800000,10,5
4,Bears,0.95,0.87,1,4000000,90,5


In [7]:
summary_df = pd.DataFrame(columns=["WaferOutput", "TotalSumOfMachines", "TotalFootprint", "TotalCostOfMachines"])
summary_df

,WaferOutput,TotalSumOfMachines,TotalFootprint,TotalCostOfMachines


In [8]:
def calc_no_of_machine_for_each_step(flow_ref_df, wafer_output):
    # function: calculate the number of machines needed for each step
    # return: (dataframe) answer_df which is a dataframe with the number of machines needed for each step
    # parameters:
        # flow_ref_df: (dataframe) a dataframe that entails the steps of each workstation and details such as RPT	LoadSize, RPTBasis, CapacityPerWeek, WaferPerBatch
        # wafer_output: (integer) desired wafer output

    # initialization
    answer_df = pd.DataFrame(columns=["WSName", "MachinesNeeded"])

    # calculating the machine number of each type
    for ws in flow_ref_df["WSName"].drop_duplicates():
        no_of_machines_total = 0
        additional_wafers_by_prev = 0
        remainder_wafer_by_prev = 0
        no_of_machines_for_this_step = 0
        time_left_after_5000_wafers = 0

        # step 1a in the report
        # ws_df contains only the steps for that particular ws
        ws_df = flow_ref_df[flow_ref_df.WSName==ws].sort_values(by="CapacityPerWeek", ascending=False).reset_index()

        # if RPT Basis is equal to 1
        if int(ws_df["RPTBasis"][0]) == 1:
            # loop through each step of the workstation
            # step 5a in the report
            for idx in range(ws_df.shape[0]):
                if idx != 0:
                    # step in between 3a and 4a (omitted) in the report
                    if additional_wafers_by_prev < wafer_output:
                        additional_wafers_by_prev = np.floor(time_left_after_5000_wafers/ws_df["RPT"][idx])*ws_df["WaferPerBatch"][idx]
                    else:
                        additional_wafers_by_prev -= wafer_output
                if additional_wafers_by_prev < wafer_output:
                    # step 2a in the report
                    no_of_machines_for_this_step = int(np.ceil((float(wafer_output)-additional_wafers_by_prev)/ws_df["CapacityPerWeek"][idx]))
                    # step 7a in the report
                    no_of_machines_total += no_of_machines_for_this_step
                    # step 3a and 4a in the report
                    time_left_after_5000_wafers = np.floor((additional_wafers_by_prev+no_of_machines_for_this_step*ws_df["CapacityPerWeek"][idx]-wafer_output)/float(ws_df["WaferPerBatch"][idx]))*ws_df["RPT"][idx]

        # if RPT Basis is more than 1
        else:
            # step 1b in the report
            # ws_df contains only the steps for that particular ws
            ws_df = flow_ref_df[flow_ref_df.WSName==ws].sort_values(by="RPT", ascending=False).reset_index()

            # loop through each step of the workstation
            # step 5b in the report
            for idx in range(ws_df.shape[0]):
                no_of_machines_for_this_step = 0
                if idx != 0:
                    # step in between 2b and 3b (omitted) in the report
                    if additional_wafers_by_prev < wafer_output:
                        additional_wafers_by_prev = np.floor(time_left_after_5000_wafers/ws_df["RPT"][idx])*ws_df["WaferPerBatch"][idx]
                    else:
                        additional_wafers_by_prev -= (wafer_output-remainder_wafer_by_prev)
                    
                    # step 3b in the report
                    remainder_wafer_by_prev = (additional_wafers_by_prev+no_of_machines_for_this_step*ws_df["CapacityPerWeek"][idx-1]-(wafer_output-remainder_wafer_by_prev))%ws_df["WaferPerBatch"][idx-1]
                
                if additional_wafers_by_prev < wafer_output:
                    # step 2b in the report
                    no_of_machines_for_this_step = int(np.ceil((float(wafer_output)-additional_wafers_by_prev)/int(ws_df["CapacityPerWeek"][idx])))
                    # step 6b in the report
                    no_of_machines_total += no_of_machines_for_this_step
                    # step 4b in the report
                    time_left_after_5000_wafers = np.floor((additional_wafers_by_prev+no_of_machines_for_this_step*ws_df["CapacityPerWeek"][idx]-(wafer_output-remainder_wafer_by_prev))/ws_df["WaferPerBatch"][idx])*ws_df["RPT"][idx]
        
        # adding calculated number of machines into a seperate dataframe
        new_row = {"WSName": ws, "MachinesNeeded": no_of_machines_total}
        answer_df = answer_df.append(new_row, ignore_index=True)

    return answer_df

In [9]:
# # qn 2.2
# answer_df = calc_no_of_machine_for_each_step(flow_ref_df=flow_df, wafer_output=5000)

# answer_df["Footprint"] = answer_df["MachinesNeeded"]*ws_data_df["FootPrint"]

# answer_df["CostOfMachine"] = answer_df["MachinesNeeded"]*ws_data_df["ToolPrice"]

# new_row ={"WaferOutput": 5000, "TotalSumOfMachines": answer_df["MachinesNeeded"].sum(), "TotalFootprint": answer_df["Footprint"].sum(), "TotalCostOfMachines":answer_df["CostOfMachine"].sum()}    
# summary_df = summary_df.append(new_row, ignore_index=True)
# summary_df

In [10]:
for wafer in range(0, 25000, 100):
    # guiding question 2.2 included in this loop
    answer_df = calc_no_of_machine_for_each_step(flow_ref_df=flow_df, wafer_output=wafer)

    # guiding question 2.4
    answer_df["Footprint"] = answer_df["MachinesNeeded"]*ws_data_df["FootPrint"]

    answer_df["CostOfMachine"] = answer_df["MachinesNeeded"]*ws_data_df["ToolPrice"]

    repo = "data_generated/answer_df_" + str(wafer) + ".csv"
    answer_df.to_csv(repo, index=False)

    new_row ={"WaferOutput": wafer, "TotalSumOfMachines": answer_df["MachinesNeeded"].sum(), "TotalFootprint": answer_df["Footprint"].sum(), "TotalCostOfMachines":answer_df["CostOfMachine"].sum()}    
    summary_df = summary_df.append(new_row, ignore_index=True)

    # just to let me know which data point the program is currently generating
    print(wafer)

# guiding question 2.3
summary_df["DeliveryTime"] = 1.5+(summary_df["TotalSumOfMachines"]/1000)**2+1.2*((summary_df["TotalSumOfMachines"]/1000)**3)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [11]:
summary_df.head()

,WaferOutput,TotalSumOfMachines,TotalFootprint,TotalCostOfMachines,DeliveryTime
0,0,0,0,0,1.5
1,100,53,5460,301600000,1.50299
2,200,53,5460,301600000,1.50299
3,300,53,5460,301600000,1.50299
4,400,53,5460,301600000,1.50299


In [12]:
summary_df.to_csv("summary.csv", index=False)